<a href="https://colab.research.google.com/github/BennettLovejoy/Portfolio/blob/main/Advanced_Folium_Portfolio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import json
import prettytable
headers = {'Content-type': 'application/json'}
data = json.dumps({"seriesid": ['CUUR0000SA0','SUUR0000SA0'],"startyear":"2011", "endyear":"2014"})
p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
json_data = json.loads(p.text)
for series in json_data['Results']['series']:
    x=prettytable.PrettyTable(["series id","year","period","value","footnotes"])
    seriesId = series['seriesID']
    for item in series['data']:
        year = item['year']
        period = item['period']
        value = item['value']
        footnotes=""
        for footnote in item['footnotes']:
            if footnote:
                footnotes = footnotes + footnote['text'] + ','
        if 'M01' <= period <= 'M12':
            x.add_row([seriesId,year,period,value,footnotes[0:-1]])
    output = open(seriesId + '.txt','w')
    output.write (x.get_string())
    output.close()

In [2]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import folium

m = folium.Map(location=(40.0675676,-83.015471), tiles="cartodb positron")
m
#m.save("index.html")


In [4]:
m = folium.Map([40.0675676,-83.015471], zoom_start=12, tiles="cartodb positron")

# Write function to get coordinates, name, relevant data from each school in XYZ dataset

folium.Marker(
    location=[40.0675676,-83.015471],
    tooltip="Click me!",
    popup="School Name",
    icon=folium.Icon(icon="cloud"),
).add_to(m)

folium.Marker(
    location=[40.0675676,-83.015471],
    tooltip="Click me!",
    popup="Current Location",
    icon=folium.Icon(color="green"),
).add_to(m)

m

In [5]:
#for i in all_coords:
    #folium.Marker(i).add_to(m)


### Bureau of Economic Analysis API Setup

[Bureau of Labor Statistics API Sample Python Code](https://www.bls.gov/developers/api_python.htm)

In [ ]:
!pip install bls

In [ ]:
import requests
import json
import prettytable
import bls

## U.S. Bureau of Labor Statistics (BLS) Application Programming Interface with Python: guided examples [website](https://www.bd-econ.com/blsapi.html)

v1

In [ ]:
# BLS API v1 url
base_url = 'https://api.bls.gov/publicAPI/v1/timeseries/data/'

In [ ]:
# BLS series id for the civilian labor force participation rate
series = {'id': 'LNS11300000',
          'name': 'Labor Force Participation Rate'}

In [ ]:
data_url = '{}{}'.format(base_url, series['id'])
print(data_url)

In [ ]:
import requests

r = requests.get(data_url).json()
print('Status: ' + r['status'])

In [ ]:
print(r.keys())

In [ ]:
r = r['Results']['series'][0]['data']
print(r[0])

In [ ]:
import pandas as pd

dates = ['{} {}'.format(i['period'], i['year']) for i in r]
index = pd.to_datetime(dates)
data = {series['id']: [float(i['value']) for i in r],
        'footnotes': [i['footnotes'][0] for i in r]}

df = pd.DataFrame(index=index, data=data).iloc[::-1]

df.tail(3)

In [ ]:
%matplotlib inline

df['mean'] = df[series['id']].mean()
df[[series['id'], 'mean']].plot(title=series['name'])

v2

In [ ]:
!pip install config

In [ ]:
import requests
import json
import prettytable
headers = {'Content-type': 'application/json'}
data = json.dumps({"seriesid": ['CUUR0000SA0','SUUR0000SA0'],"startyear":"2011", "endyear":"2014"})
p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
json_data = json.loads(p.text)
for series in json_data['Results']['series']:
    x=prettytable.PrettyTable(["series id","year","period","value","footnotes"])
    seriesId = series['seriesID']
    for item in series['data']:
        year = item['year']
        period = item['period']
        value = item['value']
        footnotes=""
        for footnote in item['footnotes']:
            if footnote:
                footnotes = footnotes + footnote['text'] + ','
        if 'M01' <= period <= 'M12':
            x.add_row([seriesId,year,period,value,footnotes[0:-1]])
    output = open(seriesId + '.txt','w')
    output.write (x.get_string())
    output.close()

## Using the Bureau of Economic Analysis Python API Library Part 1
[Youtube Video for Reference](https://www.youtube.com/watch?v=wZjA6d2WDYk)

In [ ]:
!pip install python-bea

In [ ]:
!pip install pybea configparser

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
from configparser import ConfigParser

# Initialize ConfigParser and read the uploaded configuration file
config = ConfigParser()
config.read('first_config.ini')  # The file should be in the root directory after upload

# Grab the API key from the specified section and key in the config file
API_KEY = config.get('ben_BEA_credentials', 'API_KEY')

print(API_KEY)  # Just to verify if it works (you can remove this after testing)

In [ ]:
from pprint import pprint
from configparser import ConfigParser
from pybea.client import BureauEconomicAnalysisClient

# Grab my API Key
config = ConfigParser()
config.read('configs/config.ini')

# Initialize API Client
bea_client = BureauEconomicAnalysisClient(api_key=API_KEY)

# Grab the dataset list.
#dataset_list = bea_client.get_dataset_list()
#pprint(dataset_list)


# Grab Parameters List


In [ ]:
parameters_set_list = bea_client.get_parameters_list(dataset_name='Regional')
# pprint(parameters_set_list)


## Part II of Youtube Video
[Youtube video for reference](https://www.youtube.com/watch?v=wZjA6d2WDYk)

# Grab GDP numbers for finance and insurance industry

In [ ]:
gdp_by_industry = bea_client.gdp_by_industry(
    year = 'ALL',
    industry= '52',
    frequency = ['A']
)
#pprint(gdp_by_industry)

# Grab the national product and income data


In [ ]:
national_income = bea_client.national_income_and_product_accounts(
    table_name = 'T10101',
    frequency = ['A', 'Q'],
    year = ['2010, 2012']
)
#pprint(national_income)

# Grab detailed data


In [ ]:
national_income = bea_client.national_income_and_product_accounts_detail(
    table_name = 'U20305',
    frequency = ['A', 'Q'],
    year = ['2010, 2012']
)
#pprint(national_income)

# Grab fixed assets data

In [ ]:
fixed_assets = bea_client.fixed_assets(
   table_name = 'FAAt201',
   year = ['2010, 2012']
)
#pprint(fixed_assets)


# Direct investments and multinational enterprises

In [ ]:
investments = bea_client.direct_investments_and_multinational_enterprises(
    direction_of_investment = 'outward',
    classification = 'country',
    year = '2010',
    country = ['650', '699']
)
#pprint(investments)

# Get US Direct Investment Positions

In [ ]:
investments = bea_client.activities_investments_and_multinational_enterprises(
    direction_of_investment = 'outward',
    classification = 'CountryByIndustry',
    series_id = ['4', '5'],
    year = ['2011', '2012'],
    country = '202',
    ownership_level= False,
    industry = 'ALL',
    non_bank_affilates_only= False
    )
#pprint(investments)

# Grab the Balance of Goods

In [ ]:
balance_of_goods = bea_client.international_transactions(
    indicator=['BalGds'],
    frequency=['A', 'Q'],
    year=['2010', '2012'],
    area_or_country= ['China'],
)
#pprint(balance_of_goods)

# Grab US Assets

In [ ]:
us_assets = bea_client.international_investments_positions(
    type_of_investment = ['FinAssetsExclFinDeriv'],
    component = ['ChgPosPrice'],
    year = ['ALL'],
    frequency = ['A'],
)
#pprint(us_assets)

# Input Output Data

In [ ]:
input_output_data = bea_client.input_output_statstics(
    table_id = ['56'],
    year = ['2010', '2012']
)
#pprint(input_output_data)

# Grab Underlying GDP by Industry

In [ ]:
underlying_gdp_by_industry = bea_client.underlying_gdp_by_industry(
    industry = 'ALL',
    year = ['2010', '2012'],
    frequency = ['A'],
    table_id = ['ALL']
)

#pprint(underlying_gdp_by_industry)

# Grab International Trade Services

In [ ]:
international_trade_services = bea_client.international_trade_services(
    type_of_service='AllServiceTypes',
    trade_direction=['Imports'],
    year = ['2014', '2015'],
    affiliation = ['AllAffiliations'],
    area_or_country = ['Germany'],
)
#pprint(international_trade_services)


### Bureau of Labor Statistics API Setup

[Bureau of Labor Statistics API Python Sample Code](https://www.bls.gov/developers/api_python.htm#python2)

In [ ]:
import requests
import json
import prettytable
headers = {'Content-type': 'application/json'}
data = json.dumps({"seriesid": ['CUUR0000SA0','SUUR0000SA0'],"startyear":"2011", "endyear":"2014"})
p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
json_data = json.loads(p.text)
for series in json_data['Results']['series']:
    x=prettytable.PrettyTable(["series id","year","period","value","footnotes"])
    seriesId = series['seriesID']
    for item in series['data']:
        year = item['year']
        period = item['period']
        value = item['value']
        footnotes=""
        for footnote in item['footnotes']:
            if footnote:
                footnotes = footnotes + footnote['text'] + ','
        if 'M01' <= period <= 'M12':
            x.add_row([seriesId,year,period,value,footnotes[0:-1]])
    output = open(seriesId + '.txt','w')
    output.write (x.get_string())
    output.close()